In [1]:
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers import Conv2D,GlobalMaxPooling2D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense,merge,Lambda,Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
import keras.backend as K
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D  
from keras.layers import Bidirectional, GlobalAveragePooling1D,GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
from keras.layers import Dense,merge
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
from keras.utils import multi_gpu_model
import numpy as np
import pandas as pd
import matplotlib as plot
import gc

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.clear_session()

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1732716473651071191
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22684699853
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12321255943730719310
physical_device_desc: "device: 0, name: Tesla M40 24GB, pci bus id: 0000:04:00.0, compute capability: 5.2"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 22684699853
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 4840631246511674766
physical_device_desc: "device: 1, name: Tesla M40 24GB, pci bus id: 0000:82:00.0, compute capability: 5.2"
]


In [4]:
import pickle
pat_df = pd.read_csv('patient_disease_vector.csv') #Patients

with open('400PatComboArr25.pickle', 'rb') as handle :
   combo = pickle.load(handle)

with open('NewPatReportMatrix.pickle', 'rb') as handle :
   new_vect = pickle.load(handle)

In [5]:
pat_df.head(5)

,Unnamed: 0,SUBJECT_ID,0,1,2,3,4,5,6,7,...,9,10,11,12,13,14,15,16,17,18
0,0,249,0.0,0.000000,0.003799,0.189696,0.071951,0.000000,0.013072,0.275432,...,0.093745,0.091629,0.0,0.0,0.0,0.0,0.0,0.055828,0.026500,0.102564
1,1,250,0.0,0.012821,0.217949,0.000000,0.153846,0.000000,0.000000,0.141026,...,0.000000,0.064103,0.0,0.0,0.0,0.0,0.0,0.102564,0.217949,0.000000
2,2,251,0.0,0.000000,0.000000,0.000000,0.000000,0.266667,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.400000,0.333333
3,3,252,0.0,0.031746,0.000000,0.070106,0.192857,0.047619,0.194444,0.079894,...,0.103439,0.046296,0.0,0.0,0.0,0.0,0.0,0.115873,0.022487,0.070106
4,4,253,0.0,0.000000,0.000000,0.266667,0.000000,0.133333,0.111111,0.444444,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [6]:
def get_values(batch):
    X1 = []
    X2 = []
    Y = np.empty((0,4), int)
    for i in range(len(batch)):
        p1, p2 = int(batch[i][0]) , int(batch[i][1])
        X1.append(new_vect[p1])
        X2.append(new_vect[p2])
        vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
        vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
        y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
        Y = np.vstack((Y, y))
    X1 = np.asarray(X1)
    X2 = np.asarray(X2)
    X1 = np.resize(X1,(X1.shape[0],X1.shape[1],X1.shape[2],1))
    X2 = np.resize(X2,(X2.shape[0],X2.shape[1],X2.shape[2],1))
    return [X1,X2], Y

In [7]:
from keras.utils import Sequence

class mimic_generator(Sequence):
    
    def __init__(self,combinations,batch_size):
        self.combinations = combinations
        self.batch_size = batch_size
        
    def __len__(self):
        return int(len(self.combinations)/int(self.batch_size))
    
    def __getitem__(self, index):
        batch_x = self.combinations[index * self.batch_size :(index+1) * self.batch_size]
        
        
        return  get_values(batch_x)

In [8]:
splitter = int(len(combo)*0.7)
print(splitter)

195761110


In [14]:
combinations, X_test = combo[:splitter], combo[splitter:]


In [ ]:
combinations = combinations[:1000000]
n = len(combinations)
m = int(0.9 * n)
print(m)
dim = 300
filter_size = 64
ksize = 5
num_training_samples = m
num_validation_samples = n-m

#Model1
inp1 = Input(shape=(dim,400,1))
x1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='same', kernel_initializer='normal',activation = 'relu')(inp1)

x3 = GlobalMaxPooling2D()(x1)

#Model2
inp2 = Input(shape=(dim,400,1))
y1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='same', kernel_initializer='normal', activation = 'relu')(inp2)
y3 = GlobalMaxPooling2D()(y1)

#Model3
merged = concatenate([x3,y3])
z1 = Dense(128, activation='relu')(merged)
out = Dense(4, activation="sigmoid")(z1)
model = Model(inputs=[inp1,inp2], outputs=out)
# try:
#     model = multi_gpu_model(model)
# except:
#     print("Shit")
#adm = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights-CNN3.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#history=model.fit([X11,X222],Y1,validation_split=0.3, epochs=10, batch_size=64, callbacks=callbacks_list, verbose=1)
my_training_batch_generator = mimic_generator(combinations[:m], batch_size=128)
my_validation_batch_generator = mimic_generator(combinations[m:], batch_size=64)

history = model.fit_generator(generator=my_training_batch_generator,
                    steps_per_epoch=(num_training_samples // 128),
                    epochs=5,
                    verbose=1,
                    validation_data=my_validation_batch_generator,
                    validation_steps=(num_validation_samples // 64),
                    use_multiprocessing=True,
                    workers=5,
                    max_queue_size=5,callbacks=callbacks_list)

900000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 300, 400, 64) 96064       input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 300, 400, 64) 96064       input_18[0][0]                   
__________________________________________________________________________________________________
glo

Exception in thread Thread-166:
Traceback (most recent call last):
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/site-packages/keras/utils/data_utils.py", line 548, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/site-packages/keras/utils/data_utils.py", line 522, in <lambda>
    initargs=(seqs,))
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/multiprocessing/context.py", line 118, in Pool
    context=self.get_context())
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/multiprocessing/pool.py", line 174, in __init__
    self._repopulate_pool()
  File "/home/nachiket/anaconda3/envs/myenv/lib/python3.5/multiprocessing/pool.

7031/7031 [==============================] - 43046s 6s/step - loss: 0.3677 - acc: 0.8217 - val_loss: 0.8260 - val_acc: 0.7676

Epoch 00001: val_acc improved from -inf to 0.76758, saving model to weights-CNN3.best.hdf5
Epoch 2/5
 189/7031 [..............................] - ETA: 11:11:33 - loss: 0.1357 - acc: 0.9523

In [11]:
combinations = combinations[:1000000]
n = len(combinations)
m = int(0.9 * n)
print(m)
dim = 300
filter_size = 64
ksize = 5
num_training_samples = m
num_validation_samples = n-m

#Model1
inp1 = Input(shape=(dim,400,1))
x1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='valid', kernel_initializer='normal',activation = 'relu')(inp1)
x2 = GlobalMaxPooling2D()(x1)

#Model2
inp2 = Input(shape=(dim,400,1))
y1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='valid', kernel_initializer='normal', activation = 'relu')(inp2)
y2 = GlobalMaxPooling2D()(y1)

#Model3
merged = concatenate([x2,y2])
z1 = Dense(128, activation='relu')(merged)
out = Dense(4, activation="sigmoid")(z1)
model = Model(inputs=[inp1,inp2], outputs=out)
# try:
#     model = multi_gpu_model(model)
# except:
#     print("Shit")
adm = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights-CNN.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#history=model.fit([X11,X222],Y1,validation_split=0.3, epochs=10, batch_size=64, callbacks=callbacks_list, verbose=1)
my_training_batch_generator = mimic_generator(combinations[:m], batch_size=64)
my_validation_batch_generator = mimic_generator(combinations[m:], batch_size=32)

history = model.fit_generator(generator=my_training_batch_generator,
                    steps_per_epoch=(num_training_samples // 64),
                    epochs=5,
                    verbose=1,
                    validation_data=my_validation_batch_generator,
                    validation_steps=(num_validation_samples // 32),
                    use_multiprocessing=False,
                    workers=5,
                    max_queue_size=2,callbacks=callbacks_list)

900000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 1, 396, 64)   96064       input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 1, 396, 64)   96064       input_4[0][0]                    
__________________________________________________________________________________________________
glo

In [47]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [10]:
dim = 300
filter_size = 64
ksize = 5
#Model1
inp1 = Input(shape=(dim,400,1))
x1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='valid', kernel_initializer='normal',activation = 'relu')(inp1)
x2 = GlobalMaxPooling2D()(x1)

#Model2
inp2 = Input(shape=(dim,400,1))
y1 = Conv2D(filter_size, kernel_size=(dim,ksize) ,padding='valid', kernel_initializer='normal', activation = 'relu')(inp2)
y2 = GlobalMaxPooling2D()(y1)

#Model3
merged = concatenate([x2,y2])
z1 = Dense(128, activation='relu')(merged)
out = Dense(4, activation="sigmoid")(z1)
model = Model(inputs=[inp1,inp2], outputs=out)
# try:
#     model = multi_gpu_model(model)
# except:
#     print("Shit")
adm = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights-CNN3.best.hdf5"
model.load_weights(filepath)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1, 396, 64)   96064       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 1, 396, 64)   96064       input_2[0][0]                    
__________________________________________________________________________________________________
global_max

In [10]:
X_sample_test = combinations[:10000]

In [8]:
X1 = []
X2 = []
Y = np.empty((0,4), int)

for i in range(len(X_sample_test)):
    p1, p2 = int(X_sample_test[i][0]) , int(X_sample_test[i][1])
    X1.append(new_vect[p1])
    X2.append(new_vect[p2])
    vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
    vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
    y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
    Y = np.vstack((Y, y))
X1 = np.asarray(X1)
X2 = np.asarray(X2)
X1 = np.resize(X1,(X1.shape[0],X1.shape[1],X1.shape[2],1))
X2 = np.resize(X2,(X2.shape[0],X2.shape[1],X2.shape[2],1))
    

NameError: name 'X_sample_test' is not defined

In [9]:
scores = model.evaluate([X1,X2],Y)


NameError: name 'model' is not defined

In [39]:
scores

[1.1408292901992798, 0.462475]

In [33]:
tp = [0.01 for i in range(4)]
fp = [0.01 for i in range(4)]
tn = [0.01 for i in range(4)]
fn = [0.01 for i in range(4)]

In [34]:
for i in range(len(scores)):
    for j in range(4):
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 1)):
            tp[j] += 1
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 0)):
            tn[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 0)):
            fp[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 1)):
            fn[j] += 1
                

In [35]:
print("Diseases of nervous system and sense organs precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[0]/(tp[0]+tn[0]))*100,(tp[0]/(tp[0]+fn[0]))*100,((tp[0]+tn[0])/(len(scores)))*100))
print("Diseases of circulatory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[1]/(tp[1]+tn[1]))*100,(tp[1]/(tp[1]+fn[1]))*100,((tp[1]+tn[1])/(len(scores)))*100))
print("Diseases of respiratory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[2]/(tp[2]+tn[2]))*100,(tp[2]/(tp[2]+fn[2]))*100,((tp[2]+tn[2])/(len(scores)))*100))
print("Diseases of digestive precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[3]/(tp[3]+tn[3]))*100,(tp[3]/(tp[3]+fn[3]))*100,((tp[3]+tn[3])/(len(scores)))*100))

Diseases of nervous system and sense organs precision:85.46%, recall:72.35% and accuracy:55.77%
Diseases of circulatory system precision:58.48%, recall:71.35% and accuracy:33.14%
Diseases of respiratory system precision:31.74%, recall:43.55% and accuracy:53.94%
Diseases of digestive precision:36.52%, recall:33.67% and accuracy:42.14%


In [36]:
print(scores[:10])

[[0. 0. 1. 0.]
 [1. 1. 1. 1.]
 [0. 1. 0. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 0. 0.]
 [0. 1. 1. 0.]]


In [37]:
print(Y[:10])

[[1 1 1 1]
 [1 1 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 1]
 [1 1 1 0]
 [0 1 0 0]
 [1 1 1 0]
 [1 1 0 1]
 [1 0 1 1]]


In [10]:
def get_values(batch):
    X1 = []
    X2 = []
    Y = np.empty((0,4), int)
    for i in range(len(batch)):
        p1, p2 = int(batch[i][0]) , int(batch[i][1])
        X1.append(new_vect[p1])
        X2.append(new_vect[p2])
        vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
        vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
        y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
        Y = np.vstack((Y, y))
    X1 = np.asarray(X1)
    X2 = np.asarray(X2)
    return [X1,X2], Y

In [11]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, LSTM, Dense, Dropout, Embedding, Bidirectional, GRU
from keras.optimizers import Adam
from keras import initializers, regularizers
from keras import optimizers
from keras.engine.topology import Layer
from keras import constraints

############################################## 
"""
# ATTENTION LAYER
Cite these works 
1. Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
"Hierarchical Attention Networks for Document Classification"
accepted in NAACL 2016
2. Winata, et al. https://arxiv.org/abs/1805.12307
"Attention-Based LSTM for Psychological Stress Detection from Spoken Language Using Distant Supervision." 
accepted in ICASSP 2018
Using a context vector to assist the attention
* How to use:
Put return_sequences=True on the top of an RNN Layer (GRU/LSTM/SimpleRNN).
The dimensions are inferred based on the output shape of the RNN.
Example:
	model.add(LSTM(64, return_sequences=True))
	model.add(AttentionWithContext())
	model.add(Addition())
	# next add a Dense layer (for classification/regression) or whatever...
"""
##############################################

def dot_product(x, kernel):
	"""
	Wrapper for dot product operation, in order to be compatible with both
	Theano and Tensorflow
	Args:
		x (): input
		kernel (): weights
	Returns:
	"""
	if K.backend() == 'tensorflow':
		return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
	else:
		return K.dot(x, kernel)

class AttentionWithContext(Layer):
	"""
	Attention operation, with a context/query vector, for temporal data.
	Supports Masking.
	follows these equations:
	
	(1) u_t = tanh(W h_t + b)
	(2) \alpha_t = \frac{exp(u^T u)}{\sum_t(exp(u_t^T u))}, this is the attention weight
	(3) v_t = \alpha_t * h_t, v in time t
	# Input shape
		3D tensor with shape: `(samples, steps, features)`.
	# Output shape
		3D tensor with shape: `(samples, steps, features)`.
	"""

	def __init__(self,
				 W_regularizer=None, u_regularizer=None, b_regularizer=None,
				 W_constraint=None, u_constraint=None, b_constraint=None,
				 bias=True, **kwargs):

		self.supports_masking = True
		self.init = initializers.get('glorot_uniform')

		self.W_regularizer = regularizers.get(W_regularizer)
		self.u_regularizer = regularizers.get(u_regularizer)
		self.b_regularizer = regularizers.get(b_regularizer)

		self.W_constraint = constraints.get(W_constraint)
		self.u_constraint = constraints.get(u_constraint)
		self.b_constraint = constraints.get(b_constraint)

		self.bias = bias
		super(AttentionWithContext, self).__init__(**kwargs)

	def build(self, input_shape):
		assert len(input_shape) == 3

		self.W = self.add_weight((input_shape[-1], input_shape[-1],),
								 initializer=self.init,
								 name='{}_W'.format(self.name),
								 regularizer=self.W_regularizer,
								 constraint=self.W_constraint)
		if self.bias:
			self.b = self.add_weight((input_shape[-1],),
									 initializer='zero',
									 name='{}_b'.format(self.name),
									 regularizer=self.b_regularizer,
									 constraint=self.b_constraint)

		self.u = self.add_weight((input_shape[-1],),
								 initializer=self.init,
								 name='{}_u'.format(self.name),
								 regularizer=self.u_regularizer,
								 constraint=self.u_constraint)

		super(AttentionWithContext, self).build(input_shape)

	def compute_mask(self, input, input_mask=None):
		# do not pass the mask to the next layers
		return None

	def call(self, x, mask=None):
		uit = dot_product(x, self.W)

		if self.bias:
			uit += self.b

		uit = K.tanh(uit)
		ait = dot_product(uit, self.u)

		a = K.exp(ait)

		# apply mask after the exp. will be re-normalized next
		if mask is not None:
			# Cast the mask to floatX to avoid float64 upcasting in theano
			a *= K.cast(mask, K.floatx())

		# in some cases especially in the early stages of training the sum may be almost zero and this results in NaN's. 
		# Should add a small epsilon as the workaround
		# a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
		a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

		a = K.expand_dims(a)
		weighted_input = x * a
		
		return weighted_input

	def compute_output_shape(self, input_shape):
		return input_shape[0], input_shape[1], input_shape[2]
	
class Addition(Layer):
	"""
	This layer is supposed to add of all activation weight.
	We split this from AttentionWithContext to help us getting the activation weights
	follows this equation:
	(1) v = \sum_t(\alpha_t * h_t)
	
	# Input shape
		3D tensor with shape: `(samples, steps, features)`.
	# Output shape
		2D tensor with shape: `(samples, features)`.
	"""

	def __init__(self, **kwargs):
		super(Addition, self).__init__(**kwargs)

	def build(self, input_shape):
		self.output_dim = input_shape[-1]
		super(Addition, self).build(input_shape)

	def call(self, x):
		return K.sum(x, axis=1)

	def compute_output_shape(self, input_shape):
		return (input_shape[0], self.output_dim)

In [15]:
combinations = combinations[:1000000]
n = len(combinations)
m = int(0.95 * n)
print(m)
dim = 300
filter_size = 64
num_training_samples = m
num_validation_samples = n-m

#model1
inp1 = Input(shape=(dim,400))
x2 = CuDNNGRU(64, return_sequences=True)(inp1)
x3 = AttentionWithContext()(x2)
x4 = Addition()(x3)

#model2
inp2 = Input(shape=(dim,400))
z2 = CuDNNGRU(64, return_sequences=True)(inp2)
z3 = AttentionWithContext()(z2)
z4 = Addition()(z3)

merged = concatenate([x4,z4])
y1 = Dense(256, activation='relu')(merged)
out = Dense(4, activation="sigmoid")(y1)
model = Model(inputs=[inp1,inp2], outputs=out)

adm = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights-GRU.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit model on training data
#history=model.fit([X1,X2],Y1,validation_split=0.3, epochs=10, batch_size=64, callbacks=callbacks_list, verbose=1)
my_training_batch_generator = mimic_generator(combo[:m], batch_size=64)
my_validation_batch_generator = mimic_generator(combo[m:], batch_size=32)

# model.fit_generator(generator=my_training_batch_generator,
#                     steps_per_epoch=(num_training_samples // 64),
#                     epochs=3,
#                     verbose=1,
#                     validation_data=my_validation_batch_generator,
#                     validation_steps=(num_validation_samples // 64),
#                     use_multiprocessing=True,
#                     workers=3,
#                     max_queue_size=14)
history = model.fit_generator(generator=my_training_batch_generator,
                    steps_per_epoch=(num_training_samples // 64),
                    epochs=1,
                    verbose=1,
                    
                    use_multiprocessing=True,
                    workers=5,
                    max_queue_size=14,callbacks=callbacks_list)

950000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 300, 400)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 300, 400)     0                                            
__________________________________________________________________________________________________
cu_dnngru_5 (CuDNNGRU)          (None, 300, 64)      89472       input_5[0][0]                    
__________________________________________________________________________________________________
cu_dnngru_6 (CuDNNGRU)          (None, 300, 64)      89472       input_6[0][0]                    
__________________________________________________________________________________________________
att

/home/nachiket/anaconda3/envs/myenv/lib/python3.5/site-packages/keras/callbacks.py:432: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


In [16]:
X_sample_test = combo[splitter:splitter+10000]

In [17]:
X1 = []
X2 = []
Y = np.empty((0,4), int)

for i in range(len(X_sample_test)):
    p1, p2 = int(X_sample_test[i][0]) , int(X_sample_test[i][1])
    X1.append(new_vect[p1])
    X2.append(new_vect[p2])
    vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
    vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
    y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
    Y = np.vstack((Y, y))
X1 = np.asarray(X1)
X2 = np.asarray(X2)

In [22]:
print(model.evaluate([X1,X2],Y))

10000/10000 [==============================] - 21s 2ms/step
[0.9989337594985962, 0.72815]


In [18]:
scores = model.predict([X1,X2])
for i in range(len(scores)):
    for j in range(len(scores[i])):
        if(scores[i][j] < 0.5):
            scores[i][j] = 0
        else:
            scores[i][j] = 1

In [19]:
tp = [0.01 for i in range(4)]
fp = [0.01 for i in range(4)]
tn = [0.01 for i in range(4)]
fn = [0.01 for i in range(4)]

In [20]:
for i in range(len(scores)):
    for j in range(4):
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 1)):
            tp[j] += 1
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 0)):
            tn[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 0)):
            fp[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 1)):
            fn[j] += 1
                

In [21]:
print("Diseases of nervous system and sense organs precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[0]/(tp[0]+tn[0]))*100,(tp[0]/(tp[0]+fn[0]))*100,((tp[0]+tn[0])/(len(scores)))*100))
print("Diseases of circulatory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[1]/(tp[1]+tn[1]))*100,(tp[1]/(tp[1]+fn[1]))*100,((tp[1]+tn[1])/(len(scores)))*100))
print("Diseases of respiratory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[2]/(tp[2]+tn[2]))*100,(tp[2]/(tp[2]+fn[2]))*100,((tp[2]+tn[2])/(len(scores)))*100))
print("Diseases of digestive precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[3]/(tp[3]+tn[3]))*100,(tp[3]/(tp[3]+fn[3]))*100,((tp[3]+tn[3])/(len(scores)))*100))

Diseases of nervous system and sense organs precision:70.11%, recall:20.63% and accuracy:10.07%
Diseases of circulatory system precision:75.80%, recall:99.64% and accuracy:96.16%
Diseases of respiratory system precision:62.20%, recall:95.45% and accuracy:93.06%
Diseases of digestive precision:46.09%, recall:92.82% and accuracy:91.97%


In [13]:
def get_values(batch):
    X1 = []
    X2 = []
    Y = np.empty((0,4), int)
    for i in range(len(batch)):
        p1, p2 = int(batch[i][0]) , int(batch[i][1])
        X1.append(new_vect[p1])
        X2.append(new_vect[p2])
        vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
        vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
        y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
        Y = np.vstack((Y, y))
    X1 = np.asarray(X1)
    X2 = np.asarray(X2)
    X1 = np.resize(X1,(X1.shape[0],X1.shape[1],X1.shape[2],1))
    X2 = np.resize(X2,(X2.shape[0],X2.shape[1],X2.shape[2],1))
    return [X1,X2], Y

In [11]:
from keras.applications.resnet50 import ResNet50


resnet50 = ResNet50(weights=None ,include_top = False,input_shape = (300,600,1))

inp1 = Input(shape=(300,400,1))
x1 = resnet50(inp1)
x2 = Flatten()(x1)

inp2 = Input(shape=(300,400,1))
y1 = resnet50(inp2)
y2 = Flatten()(y1)

merged = concatenate([x2,y2])
z1 = Dense(256, activation='relu')(merged)
out = Dense(4, activation="softmax")(z1)
model = Model(inputs=[inp1,inp2], outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/home/nachiket/anaconda3/envs/myenv/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 300, 400, 1)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                multiple             23581440    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 266240)       0           resnet50[1][0]                   
__________

In [14]:
combinations = combo[:1000000]
n = len(combinations)
m = int(0.90 * n)
print(m)
dim = 300
filter_size = 64
ksize = 5
num_training_samples = m
num_validation_samples = n-m

#Checkpoint
filepath="weights-ResCNN.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#history=model.fit([X11,X222],Y1,validation_split=0.3, epochs=10, batch_size=64, callbacks=callbacks_list, verbose=1)
my_training_batch_generator = mimic_generator(combinations[:m], batch_size=16)
my_validation_batch_generator = mimic_generator(combinations[m:], batch_size=16)

history = model.fit_generator(generator=my_training_batch_generator,
                    steps_per_epoch=(num_training_samples // 64),
                    epochs=10,
                    verbose=1,
                    validation_data=my_validation_batch_generator,
                    validation_steps=(num_validation_samples // 32),
                    use_multiprocessing=False,
                    workers=5,
                    max_queue_size=2,callbacks=callbacks_list)

900
Epoch 1/10
14/14 [==============================] - 44s 3s/step - loss: 17.0130 - acc: 0.6652 - val_loss: 19.8118 - val_acc: 0.7917

Epoch 00001: val_acc improved from -inf to 0.79167, saving model to weights-ResCNN.best.hdf5
Epoch 2/10
14/14 [==============================] - 17s 1s/step - loss: 17.6292 - acc: 0.6920 - val_loss: 19.8118 - val_acc: 0.7917

Epoch 00002: val_acc did not improve from 0.79167
Epoch 3/10
14/14 [==============================] - 17s 1s/step - loss: 17.4133 - acc: 0.7009 - val_loss: 19.8118 - val_acc: 0.7917

Epoch 00003: val_acc did not improve from 0.79167
Epoch 4/10
14/14 [==============================] - 17s 1s/step - loss: 19.4281 - acc: 0.7589 - val_loss: 19.8118 - val_acc: 0.7917

Epoch 00004: val_acc did not improve from 0.79167
Epoch 5/10
14/14 [==============================] - 17s 1s/step - loss: 16.8377 - acc: 0.7589 - val_loss: 19.8118 - val_acc: 0.7917

Epoch 00005: val_acc did not improve from 0.79167
Epoch 6/10
14/14 [====================

In [15]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [16]:
from keras.applications.resnet50 import ResNet50


resnet50 = ResNet50(weights=None ,include_top = False,input_shape = (300,600,1))

inp1 = Input(shape=(300,400,1))
x1 = resnet50(inp1)
x2 = Flatten()(x1)

inp2 = Input(shape=(300,400,1))
y1 = resnet50(inp2)
y2 = Flatten()(y1)

merged = concatenate([x2,y2])
z1 = Dense(256, activation='relu')(merged)
out = Dense(4, activation="softmax")(z1)
model = Model(inputs=[inp1,inp2], outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights(filepath)

/home/nachiket/anaconda3/envs/myenv/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [17]:
X1 = []
X2 = []
Y = np.empty((0,4), int)

for i in range(len(X_sample_test)):
    p1, p2 = int(X_sample_test[i][0]) , int(X_sample_test[i][1])
    X1.append(new_vect[p1])
    X2.append(new_vect[p2])
    vec1 = pat_df.loc[pat_df['SUBJECT_ID']==p1].iloc[0,8:12].values
    vec2 = pat_df.loc[pat_df['SUBJECT_ID']==p2].iloc[0,8:12].values
    y = np.logical_not(np.logical_xor(vec1,vec2)).astype(int)
    Y = np.vstack((Y, y))
X1 = np.asarray(X1)
X2 = np.asarray(X2)
X1 = np.resize(X1,(X1.shape[0],X1.shape[1],X1.shape[2],1))
X2 = np.resize(X2,(X2.shape[0],X2.shape[1],X2.shape[2],1))

In [18]:
scores = model.predict([X1,X2])
for i in range(len(scores)):
    for j in range(len(scores[i])):
        if(scores[i][j] < 0.5):
            scores[i][j] = 0
        else:
            scores[i][j] = 1

In [19]:
tp = [0.01 for i in range(4)]
fp = [0.01 for i in range(4)]
tn = [0.01 for i in range(4)]
fn = [0.01 for i in range(4)]

In [20]:
for i in range(len(scores)):
    for j in range(4):
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 1)):
            tp[j] += 1
        if ((scores[i][j] == Y[i][j]) & (Y[i][j] == 0)):
            tn[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 0)):
            fp[j] += 1
        if ((scores[i][j] != Y[i][j]) & (Y[i][j] == 1)):
            fn[j] += 1
                

In [21]:
print("Diseases of nervous system and sense organs precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[0]/(tp[0]+tn[0]))*100,(tp[0]/(tp[0]+fn[0]))*100,((tp[0]+tn[0])/(len(scores)))*100))
print("Diseases of circulatory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[1]/(tp[1]+tn[1]))*100,(tp[1]/(tp[1]+fn[1]))*100,((tp[1]+tn[1])/(len(scores)))*100))
print("Diseases of respiratory system precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[2]/(tp[2]+tn[2]))*100,(tp[2]/(tp[2]+fn[2]))*100,((tp[2]+tn[2])/(len(scores)))*100))
print("Diseases of digestive precision:{:.2f}%, recall:{:.2f}% and accuracy:{:.2f}%".format((tp[3]/(tp[3]+tn[3]))*100,(tp[3]/(tp[3]+fn[3]))*100,((tp[3]+tn[3])/(len(scores)))*100))

Diseases of nervous system and sense organs precision:100.00%, recall:100.00% and accuracy:34.22%
Diseases of circulatory system precision:0.00%, recall:0.00% and accuracy:26.85%
Diseases of respiratory system precision:0.00%, recall:0.00% and accuracy:39.36%
Diseases of digestive precision:0.00%, recall:0.00% and accuracy:54.33%
